##Gibbs sampling

###P(High Income | Payment = payback)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security

def mrv(index,evidences,non_evidence):
    if index==0:
        x=non_evidence.pop(0)
        x=non_evidence.pop(1)



    if index==1:
        x=non_evidence.pop(0)
        x=non_evidence.pop(0)

    if index==2:
        x=non_evidence.pop(1)
        x=non_evidence.pop(1)

    if index==3:
        x=non_evidence.pop(0)
        x=non_evidence.pop(1)
    return evidences,non_evidence


def gibbs(num_samples, evidence):
    total_weight = 0
    true_weight = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample =  random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample =  random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        non_evidence=[income_prob[income_sample],housing_prob[housing_sample],
                      deposit_prob[income_sample][deposit_sample] ,security_prob[(housing_sample,payment_sample)][security_sample]]
        evidences=[payment_prob[(income_sample,deposit_sample)][0] ]
        index = random.randint(0, len(non_evidence) - 1)
        x=non_evidence.pop(index)
        evidences,non_evidence=mrv(index,evidences,non_evidence)
        probability=0
        if index==0:
            probability=income_prob[(income_sample+1)%2]
        elif index==1:
            probability=housing_prob[(housing_sample+1)%2]
        elif index==2:
            probability=deposit_prob[income_sample][(deposit_sample+1)%2]
        else:
            probability=security_prob[(housing_sample,payment_sample)][(security_sample+1)%2]

        alpha=1
        for i in non_evidence:
            alpha*=i
        for i in evidences:
            alpha*=i

        alpha0=(alpha*x)/((alpha*x)+(alpha*probability))
        alpha1=1-alpha0

        total_weight +=alpha0+alpha1

        if income_sample == 1 :
            true_weight += alpha0
        else:
            true_weight+=alpha1

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  true_weight / total_weight
    return probability


evidence = {'Payment': 0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = gibbs(num_samples, evidence)
print(result)



0.4519340000000265


###Likelihood sampling

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security

def mrv(index,evidences,non_evidence):
    if index==0:
        x=non_evidence.pop(0)
        x=evidences.pop(0)

    if index==1:
        x=non_evidence.pop(1)
        x=evidences.pop(1)

    if index==2:
        pass


    return evidences,non_evidence


def gibbs(num_samples, evidence):
    total_weight = 0
    true_weight = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample =  random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample =  random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        non_evidence=[housing_prob[housing_sample],
                      deposit_prob[income_sample][deposit_sample]
                      ,payment_prob[(income_sample,deposit_sample)][payment_sample]]
        evidences=[income_prob[0],security_prob[(housing_sample,payment_sample)][0] ]

        index = random.randint(0, len(non_evidence) - 1)
        x=non_evidence.pop(index)
        evidences,non_evidence=mrv(index,evidences,non_evidence)
        probability=0
        if index==0:
            probability=housing_prob[(housing_sample+1)%2]
        elif index==1:
            probability=deposit_prob[income_sample][(deposit_sample+1)%2]


        else:
            probability=payment_prob[(income_sample,deposit_sample)][(payment_sample+1)%2]

        alpha=1
        for i in non_evidence:
            alpha*=i
        for i in evidences:
            alpha*=i

        alpha0=(alpha*x)/((alpha*x)+(alpha*probability))
        alpha1=1-alpha0

        total_weight +=alpha0+alpha1

        if income_sample == 1 :
            true_weight += alpha0
        else:
            true_weight+=alpha1

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  true_weight / total_weight
    return probability


evidence = {'Payment': 0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = gibbs(num_samples, evidence)
print(result)



0.5206649999999934


###P(Payment = default | Deposit = Large)


In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security

def mrv(index,evidences,non_evidence):
    if index==0:
        x=non_evidence.pop(0)
        x=evidences.pop(0)

    if index==1:
        x=non_evidence.pop(0)
        x=non_evidence.pop(0)

    if index==2:
        x=non_evidence.pop(1)
        x=evidences.pop(0)


    return evidences,non_evidence


def rejection_sampling(num_samples, evidence):
    total_weight = 0
    true_weight = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample =  random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample =  random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        non_evidence=[housing_prob[housing_sample],
                      income_prob[income_sample],
                         security_prob[(housing_sample,payment_sample)][security_sample],
                      payment_prob[(income_sample,deposit_sample)][payment_sample]]
        evidences=[deposit_prob[income_sample][1]]-

        index = random.randint(0, len(non_evidence) - 1)
        x=non_evidence.pop(index)
        evidences,non_evidence=mrv(index,evidences,non_evidence)
        probability=0
        if index==0:
            probability=housing_prob[(housing_sample+1)%2]
        elif index==1:
            probability=income_prob[(income_sample+1)%2]
        elif index==2:
            probability=security_prob[(housing_sample,payment_sample)][(security_sample+1)%2]
        else:
            probability=payment_prob[(income_sample,deposit_sample)][(payment_sample+1)%2]

        alpha=1
        for i in non_evidence:
            alpha*=i
        for i in evidences:
            alpha*=i

        alpha0=(alpha*x)/((alpha*x)+(alpha*probability))
        alpha1=1-alpha0

        total_weight +=alpha0+alpha1

        if income_sample == 1 :
            true_weight += alpha0
        else:
            true_weight+=alpha1

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  true_weight / total_weight
    return probability


evidence = {'Payment': 0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = rejection_sampling(num_samples, evidence)
print(result)



0.43277300000001934


##Likelihood sampling

##P(High Income | Payment = payback)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security



def likelihood_sampling(num_samples, evidence):
    total_weight = 0
    true_weight = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample = random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample = random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        # Step 3: Check if the evidence matches the observed evidence
        if (evidence['Payment'] == payment_sample):

            total_weight += payment_prob[(income_sample,deposit_sample)][payment_sample]*income_prob[income_sample]

            if income_sample == 1 :
                true_weight +=income_prob[income_sample]*payment_prob[(income_sample,deposit_sample)][payment_sample]

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  true_weight / total_weight
    return probability


evidence = {'Payment': 0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = likelihood_sampling(num_samples, evidence)
print(result)



0.1920012305799133


##P(Housing = Tenant | low income,  No security)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security



def  likelihood_sampling(num_samples, evidence):
    total_weight = 0
    true_weight = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample = random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample = random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        # Step 3: Check if the evidence matches the observed evidence
        if (evidence['Income'] == income_sample and evidence['Security']==security_sample):
            total_weight += income_prob[income_sample] * security_prob[(housing_sample,payment_sample)][0]*housing_prob[housing_sample]

            if housing_sample == 1 :
                true_weight += housing_prob[housing_sample]*income_prob[income_sample] * security_prob[(housing_sample,payment_sample)][0]

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  true_weight / total_weight
    return probability


evidence = {'Income': 0, 'Security':0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result =  likelihood_sampling(num_samples, evidence)
print(result)



0.28950098616325814


## P(Payment = default | Deposit = Large)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security



def  likelihood_sampling(num_samples, evidence):
    total_weight = 0
    true_weight = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample = random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample = random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        # Step 3: Check if the evidence matches the observed evidence
        if (evidence['Deposit'] == deposit_sample ):
            total_weight += deposit_prob[income_sample][1]*payment_prob[(income_sample,deposit_sample)][1]
            if payment_sample == 1 :
                true_weight+= payment_prob[(income_sample,deposit_sample)][1]*deposit_prob[income_sample][1]

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  true_weight / total_weight
    return probability


evidence = {'Deposit': 1}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result =  likelihood_sampling(num_samples, evidence)
print(result)



0.46001084752997345


##Rejection sampling

### P(High Income | Payment = payback)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security



def rejection_sampling(num_samples, evidence):
    total_valid_samples = 0
    numerator_true_samples = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample = random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample = random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        # Step 3: Check if the evidence matches the observed evidence
        if (evidence['Payment'] == payment_sample):
            total_valid_samples += 1
            if income_sample == 1 :
                numerator_true_samples+=1

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  numerator_true_samples / total_valid_samples
    return probability


evidence = {'Payment': 0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = rejection_sampling(num_samples, evidence)
print(result)



0.3300434267666798


###P(Housing = Tenant | low income,  No security)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security



def rejection_sampling(num_samples, evidence):
    total_valid_samples = 0
    numerator_true_samples = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample = random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample = random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        # Step 3: Check if the evidence matches the observed evidence
        if (evidence['Income'] == income_sample and evidence['Security']==security_sample):
            total_valid_samples += 1
            if housing_sample == 1 :
                numerator_true_samples+=1

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  numerator_true_samples / total_valid_samples
    return probability


evidence = {'Income': 0, 'Security':0}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = rejection_sampling(num_samples, evidence)
print(result)



0.364021662349894


###P(Payment = default | Deposit = Large)

In [ ]:
import random

income_prob =  {0:0.7 ,  1:0.3} # 0 indicates low income , 1 indicates high income
housing_prob = {0:0.65,  1:0.35} #0 indicates tenant and 1 indictes real_estate
deposit_prob = {0:{0:0.4, 1:0.6} , 1:{0:0.9, 1:0.1}} # key zero indicates income, in the inner dic, 0,1 inc=dicate small deposit and large deposit
payment_prob = {(0,0):{0:0.4 , 1:0.6} , (0,1):{0:0.55 , 1:0.45} , (1,0):{0:0.5 , 1:0.5} , (1,1):{0:0.95 , 1:0.05}} # outer key (x,y) --> (Income ,Deposit), inner dic 0,1 indicte payback, default
security_prob = {(0,0):{0:0.69 , 1:0.31}, (0,1):{0:0.5 , 1:0.5}, (1,0):{0: 0.25, 1:0.75} , (1,1):{0:0.99 , 1:0.01}} # outer key (x,y) --> (Housing, Payment), inner dic 0,1 indicate no security, security



def rejection_sampling(num_samples, evidence):
    total_valid_samples = 0
    numerator_true_samples = 0

    for _ in range(num_samples):
        # Step 1: Generate a sample for income and housing
        income_sample = random.choices([0, 1], [income_prob[0], income_prob[1]])[0]
        housing_sample = random.choices([0,1], [housing_prob[0], housing_prob[1]])[0]

        # Step 2: Given the sample, generate samples for deposit, payment, security
        deposit_sample = random.choices([0, 1], [deposit_prob[income_sample][0], deposit_prob[income_sample][1]])[0]
        payment_sample = random.choices([0, 1], [payment_prob[(income_sample,deposit_sample)][0], payment_prob[(income_sample,deposit_sample)][1]])[0]
        security_sample = random.choices([0, 1],[security_prob[(housing_sample,payment_sample)][0], security_prob[(housing_sample,payment_sample)][1]])[0]

        # Step 3: Check if the evidence matches the observed evidence
        if (evidence['Deposit'] == deposit_sample ):
            total_valid_samples += 1
            if payment_sample == 1 :
                numerator_true_samples+=1

    # Estimate the probability by dividing the valid samples by the total samples
    probability =  numerator_true_samples / total_valid_samples
    return probability


evidence = {'Deposit': 1}

# Perform rejection sampling with 10000 samples
num_samples = 10000
result = rejection_sampling(num_samples, evidence)
print(result)



0.41839762611275966
